# Notebook 2: Velocity Interneurons

### General Overview

Velocity interneurons (IN) are modeled to spike whenever the joint angle transitions from the receptive field of one hair to the next. To accomplish this, we could leverage the phasic response of the sensory neurons. Connecting a high-pass filter with the correct paramaters to each hair ensures that only the spikes from the phasic peak pass through the filter. All the high-pass filter outputs of a hair field combine into a velocity IN. Since the hair field is binary, there are two velocity IN per joint. One encoding positive movement, with the other encoding negative movement.

### Cell-by-Cell Description

#### Importing Modules and Creating Folders

This cell serves to import general functions, modules, and dictionaries from the 'general' module. Additionally, it imports the LIF, AdEx and HairField classes, which is integral to subsequent analyses. See 'classes.py' for more information.

In [ ]:
from general import *
from classes import *

create_folder_if_not_exists('images')
create_folder_if_not_exists('images/velocity_neuron')
create_folder_if_not_exists('results')

### The LIF Model

The Leaky Integrate-and-Fire (LIF) model establishes a linear relationship between the input current or spikes and the resulting output spike rate. Unlike the AdEx model, which includes adaptation mechanisms, the LIF model solely integrates its input and decays over time without any adaptation. The LIF model dynamics are implemented using the `LIF` class from the 'classes.py' file. 

First, we define the model parameters, create an instance, and initialize the model. We need two kinds of LIF models: a high pass filter and a velocity input (IN). The high pass filter can be constructed from the LIF while carefully balancing the synaptic weight ω and time constant τ The velocity IN combines all input spikes, creating a postsynaptic spike at each presynaptic spike. To achieve this, we ensure that the synaptic weight ω is greater than the threshold potential minus the resting potential, i.e., $\omega > V_T - E_L$   

In [ ]:
# Define high-pass filter parameters
hp_filter_parameters = {
    'tau': 5e-3,                                                # Membrane time constant (s)
    'V_T': -50e-3,                                              # threshold voltage (V)
    'V_R': -70e-3,                                              # Reset voltage after spike (V)
    'n': 2 * constants['N_ANGLES'] * parameters['N_HAIRS'],     # Number of neurons (1 for each hair and joint)
    'w': 10.8e-3,                                               # Synaptic weight
    'N_input': 1,                                               # Number of input neurons
    'dt': constants['dt'],                                      # Simulation time step (s)
    'n_sims': 1,                                                # Number of simulations
    'multiple_synapses': False                                  # Multiple synapses flag
}

# Define velocity parameters
velocity_parameters = {
    'tau': 5e-3,                                                
    'V_T': -50e-3,
    'V_R': -70e-3,
    'n': 2 * constants['N_ANGLES'],     # Number of neurons (2 for each joint)
    'w': 30e-3,
    'N_input': parameters['N_HAIRS'],
    'dt': constants['dt'],
    'refrac': 0,
    'n_sims': 1,
    'multiple_synapses': True
}

# Initialize the high pass filter
hp_filter = LIF(hp_filter_parameters)
hp_filter.initialize_state()

# Initialize velocity neuron
velocity_neuron = LIF(velocity_parameters)
velocity_neuron.initialize_state()

### The Velocity Neuron

The 'run_velocity()' function processes spike sensory data across multiple simulations and time steps. It applies a high-pass filter to the sensory input, reshapes the filtered output, and computes the velocity neuron spike behaviour, storing the results in a structured array.

In [ ]:
def run_velocity(spike_sensory, n_simulations, n_steps):
    
    # Initialize constants and arrays
    n_angles = constants['N_ANGLES']
    n_hairs = parameters['N_HAIRS']
    spike_sensory_filtered = np.empty((n_steps, hp_filter_parameters['n']), dtype=np.uint8)
    spike_velocity = np.empty((n_steps, 2 * n_angles, n_simulations), dtype=np.uint8)
    
    # Loop over all simulations
    for k in tqdm(range(n_simulations)):
        # Extract kth spike train to work with smaller arrays
        spike_sensory_k = spike_sensory[:, :, k]
        
        for i in range(n_steps):
            # Perform high pass filter on the current step
            _, filtered_output = hp_filter.forward(spike_sensory_k[i, :])
            spike_sensory_filtered[i, :] = filtered_output
            
            # Reshape the filtered output and combine spikes to get velocity estimate
            reshaped_filtered_output = filtered_output.reshape(2 * n_angles, n_hairs)
            _, velocity_output = velocity_neuron.forward(reshaped_filtered_output)
            spike_velocity[i, :, k] = velocity_output
    
    return spike_velocity

This cell loads the sensory spike data for both step and no-step trials. It then processes this data using the run_velocity() function and saves the resulting output spikes.

In [ ]:
# Load sensory spike data from files
with open('temp_data/spike_sensory', 'rb') as file:
    spike_sensory = np.load(file) 
    
with open('temp_data/spike_sensory_step', 'rb') as file:
    spike_sensory_step = np.load(file) 

# Run velocity neuron simulation for regular and step data
spike_velocity =  run_velocity(spike_sensory, np.min((parameters['N_SIMULATIONS'], 78)), constants['N_STEPS'])
spike_velocity_step =  run_velocity(spike_sensory_step, np.min((parameters['N_SIMULATIONS'], 21)), constants['N_STEPS_STEP'])

# Save the velocity spike data if required
if parameters['save_data']:
    with open('temp_data/spike_velocity', 'wb') as file:
        np.save(file, spike_velocity)
        
    with open('temp_data/spike_velocity_step', 'wb') as file:
        np.save(file, spike_velocity_step)

### Statistics and Performance

In this cell, the firing rate is computed from the spike train arrays. This firing rate is then z-normalized and visually compared to the z-normalized experimental joint velocities. From this comparison, the Mean Squared Error (MSE) is calculated for each timestep, trial, and joint. The process is repeated with a time delta of 0.025s to demonstrate that the response to the velocity input is delayed by 0.025s.

In [ ]:
# Load joint angles from file
with open('temp_data/joint_angles', 'rb') as file:
    joint_angles = np.load(file)

# Define deltas to show that the velocity response is 0.025ms delayed
delta_1 = int(0.025 / constants['dt'])
delta_2 = 1

# Initialize list to store mean squared errors
MSEs = []

# Define time vector
time = np.linspace(0, constants['T_TOTAL'], num=constants['N_STEPS'])

# Loop over deltas
for delta in [delta_1, delta_2]:
    MSE = np.zeros((parameters['N_SIMULATIONS'], constants['N_ANGLES']))
    
    # Loop over the number of simulations and 18 joints
    for k in range(parameters['N_SIMULATIONS']):
        for m in range(constants['N_ANGLES']):
            
            # Compute firing rates for vel- and vel+
            firing_rate_down = get_firing_rate_convolve(spike_velocity[:, 2*m, k], constants['dt'], t=0.01, nan_bool=False)
            firing_rate_up = get_firing_rate_convolve(spike_velocity[:, 2*m + 1, k], constants['dt'], t=0.01, nan_bool=False)
            
            # Combine the firing rates, vel+ - vel-
            combined_firing_rate = firing_rate_up - firing_rate_down
            
            # Z-Normalize the combined firing rate and joint angles
            combined_firing_rate_norm = zscore.zscore(combined_firing_rate)
            joint_velocity_norm = zscore.zscore(np.diff(joint_angles[:, m, k]) / constants['dt'])
            
            # Calculate MSE in each iteration, skip the first 2000 (to compensate for the delta)
            MSE[k, m] = np.mean((combined_firing_rate_norm[2001:] - joint_velocity_norm[2000-delta: -delta])**2)
            
            # Plot only the first leg (R1) and joint (alpha)
            if k == 0 and m == 0:
                # Create subplots
                fig, ax = plt.subplots()
                
                # Plot the normalized joint velocity and model prediction 
                ax.plot(time[:-1], joint_velocity_norm, color='black', label='Exp. data')
                ax.plot(time[:-delta], combined_firing_rate_norm[delta:], color=custom_colors[0], label='Model response')
                
                # Set axis parameters
                ax.set_xlim([1, 3])
                ax.set_xlabel("Time (s)")
                ax.set_ylabel("Angular velocity (a.u.)")
                ax.minorticks_on()
                ax.legend(frameon=False)
                
                # pad fig and save
                fig.tight_layout(pad=parameters['pad'])
                fig.savefig('images/velocity_neuron/P1_fig5d.png')
       
    # Append the mean MSE of the delta
    MSEs.append(np.mean(MSE))

# Print MSEs
print(MSEs)

This cell processes joint angle data to compute joint velocities and evaluate the performance of the velocity spike trains against the experimental data. It calculates binary arrays for upward and downward joint velocities, computes confusion matrices for the positive and negative velocity spike trains, and derives statistical measures from these matrices. These measures, along with pre-calculated Mean Squared Errors (MSEs), are compiled into a DataFrame and saved as a CSV file.

In [ ]:
# Load joint angle data from file
with open('temp_data/joint_angles', 'rb') as file:
    joint_angles = np.load(file)

# Compute joint velocity by taking the difference along the first (time) axis
joint_velocity = np.diff(joint_angles[:, :, :parameters['N_SIMULATIONS']], axis=0)

# Generate binary arrays for positive (1) and negative (0) joint velocities
joint_velocity_positive = np.heaviside(joint_velocity, 0)
joint_velocity_negative = 1 - joint_velocity_positive

# Compute confusion matrices for the positive and negative velocity spike trains vs. the experimental data
true_positive, false_positive, _, _ = get_confusion_matrix(spike_velocity[1:, 1::2], joint_velocity_positive)
true_negative, false_negative, _, _ = get_confusion_matrix(spike_velocity[1:, 0::2], joint_velocity_negative)

# Get statistical measures from confusion matrices and append MSEs (calculated in previous cell)
statistical_data = get_statistics(true_positive, true_negative, false_positive, false_negative)
statistical_data += MSEs

# Define labels for the statistical data
statistical_data_labels = [
    'true positive', 'true negative', 'false positive', 'false negative', 
    'positives', 'negatives', 'total', 'MCC', 'accuracy', 'TPR', 'TNR', 
    'MSE_corrected', 'MSE_not_corrected'
]

# Create a DataFrame from the statistical data and labels
df = pd.DataFrame({'value': statistical_data}, index=statistical_data_labels)

# Save the DataFrame to a CSV file
df.to_csv('results/velocity_accuracy_table.csv')

### Figures

This cell plots the vel+ and vel- spiking frequency as a function of time as well as the joint velocity for leg R1, joint α, and trial 1. 

In [ ]:
# Extract the first simulation from the joint angles data
joint_angles = joint_angles[:, :, 0]

# Create a figure with two y-axes sharing the same x-axis
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

# Calculate firing rates for downward and upward spike velocities
firing_rate_down = get_firing_rate_convolve(spike_velocity[:, 0, 0], constants['dt'], t=0.025)
firing_rate_up = get_firing_rate_convolve(spike_velocity[:, 1, 0], constants['dt'], t=0.025)

# Plot the experimental joint angular velocity and horizontal dotted line (y=0) on the first y-axis
ax1.plot(time[1:], np.diff(joint_angles[:, 0]) / constants['dt'], color='black', linestyle=custom_linestyles[0], label='Exp. data')
ax1.plot(time, np.full(time.size, 0), color='black', linestyle='dotted')

# Plot the firing rates on the second y-axis
ax2.plot(time, firing_rate_down, color=custom_colors[0], label='vel-')
ax2.plot(time, firing_rate_up, color=custom_colors[1], label='vel+')

# Set the x-axis limits and labels
ax1.set_xlim([1, 2])
ax1.set_xlabel("Time (s)")

# Set the y-axis labels
ax1.set_ylabel("Angular velocity (°$\cdot$s$^{-1}$)")
ax2.set_ylabel("Spike rate (Hz)")

# Enable minor ticks on both axes
ax1.minorticks_on()
ax2.minorticks_on()

# Add a legend to the figure
fig.legend(loc='upper left', bbox_to_anchor=[0.58, 0.97], frameon=False)

# Save the figure
fig.tight_layout(pad=parameters['pad'])
fig.savefig('images/velocity_neuron/P1_fig5c.png')

This cell plots the joint angles and overlays the individual spikes of the vel+ and vel- neurons for leg R1, joint α, and trial 1.

In [ ]:
# Convert spike_velocity array to float32 and replace zeros with NaNs for visualization
spike_velocity = spike_velocity.astype(np.float32)
spike_velocity[spike_velocity == 0] = np.nan

# Create a figure and axis for plotting
fig, ax = plt.subplots()

# Plot the experimental joint angles over time
plt.plot(time, joint_angles[:, 0], color='black')

# Scatter plot for joint angles with vel- and vel+ spikes overlayed
plt.scatter(time, joint_angles[:, 0] * spike_velocity[:, 0, 0], color=custom_colors[0], marker=custom_markers[0], s=10)
plt.scatter(time, joint_angles[:, 0] * spike_velocity[:, 1, 0], color=custom_colors[1], marker=custom_markers[1], s=10)

# Set the x-axis limits and labels
ax.set_xlim([1, 5])
ax.set_xlabel("Time (s)")

# Set the y-axis label
ax.set_ylabel("Joint angle (°)")

# Enable minor ticks on the axis
ax.minorticks_on()

# Add a legend to the plot
ax.legend(['Exp. data', 'vel-', 'vel+'], frameon=False, loc='upper center')

# Save the figure
fig.tight_layout(pad=parameters['pad'])
fig.savefig('images/velocity_neuron/P1_fig5b.png')

This cell plots the LIF response using the velocity parameters to a 333 Hz spike train

In [ ]:
# Set up subplots with specific size and layout
fig, [ax, ax2, ax3] = plt.subplots(3, figsize=(3.75*0.95, 1.2*0.95*3.75/1.61803398875), 
                                   gridspec_kw={'height_ratios': [1, 2.5, 1]}, sharex='all')
plt.subplots_adjust(wspace=0, hspace=0.05)  # Adjust space between subplots

# Change multiple synapses to false in the velocity parameters (there is one input)
hp_filter_parameters['multiple_synapses'] = False
hp_filter_parameters['n'] = 1

# Initialize the LIF neuron with velocity parameters
LIF_neuron = LIF(hp_filter_parameters)
LIF_neuron.initialize_state()

# Create input signal
input = np.zeros(1000)
input[50:800][::12] = 1  # Set spikes in input

# Initialize arrays to store voltage, spikes, and time
voltage, spikes, time = np.zeros(input.size), np.zeros(input.size), np.zeros(input.size)

# Simulate the neuron dynamics
for i in range(input.size):
    voltage[i], spikes[i] = LIF_neuron.forward(input[i])
    time[i] = i * 0.0001  # Update time

# Plot the input spikes
ax.plot(time, input, color=custom_colors[0])    
ax.set_ylabel("Pre")  # Label for the input spikes subplot
ax.set_yticks([0, 1])     # Y-axis ticks for input spikes
ax.set_xticks([0, 0.1, 0.2])  # X-axis ticks for time
ax.set_ylim(0, 1.5)       # Set Y-axis limits

# Plot the membrane potential
ax2.plot(time, voltage * 1000, color=custom_colors[0])
ax2.set_ylabel("V (mV)")  # Label for the membrane potential subplot
ax2.set_yticks([-70, -50])  # Y-axis ticks for membrane potential

# Plot the output spikes
ax3.plot(time, spikes, color=custom_colors[0])
ax3.set_ylabel("Post")  # Label for the output spikes subplot
ax3.set_yticks([0, 1])     # Y-axis ticks for output spikes
ax3.set_xlabel("Time (s)")  # X-axis label for time
ax3.set_ylim(0, 1.5)       # Set Y-axis limits

# Align Y-axis labels of all subplots
fig.align_ylabels([ax, ax2, ax3])

# Adjust layout to prevent overlap
fig.tight_layout(pad=parameters['pad'])

# Save the figure
fig.savefig('images/velocity_neuron/P1_S2.png')

This cell plots the velocity neuron spike rate as a function of angular velocity for various hair numbers (hair field) and synaptic weights (high pass filter).

In [ ]:
# Check if optimization is to be run
if parameters['run_optimization']:
    # Define parameters
    N_HAIRS_LIST = np.array([25, 50])
    WEIGHT_LIST = np.array([9e-3, 10.8e-3])
    I_phi = 50e-12
    text_offset = [0, +30, -30, 0]
    N_SPEEDS = 11
    T_TOTAL = 20
    index = 0  
    
    # Calculate the number of time steps and time array
    N_STEPS = int(T_TOTAL / constants['dt'])
    time = np.linspace(0, constants['T_TOTAL'], num=constants['N_STEPS'])
    
    # Create the speeds array and ramp durations
    speeds = np.linspace(25, 300, num=N_SPEEDS)
    ramp = (180 / (speeds * constants['dt'])).astype(int)
    speeds = np.concatenate((np.zeros(1), speeds))
    
    # Initialize arrays to store spike velocities and spike rates
    spike_velocity = np.zeros((N_STEPS, N_SPEEDS))
    spike_rates = np.zeros((N_HAIRS_LIST.size, WEIGHT_LIST.size, N_SPEEDS + 1))
    
    # Create a figure and axis for plotting
    fig, ax = plt.subplots()
    
    # Loop through combinations of hair counts and weights
    for l, m in tqdm(np.ndindex(N_HAIRS_LIST.size, WEIGHT_LIST.size)):
        
        # Define hair field parameters
        parameters_hair_field = {
            'N_hairs': N_HAIRS_LIST[l],
            'max_joint_angle': 180,
            'min_joint_angle': 0,
            'max_angle': 90,
            'overlap': 4
        }

        # Define sensory neuron parameters
        sensory_parameters = {
            'C': 200e-12,
            'g_L': 2e-9,
            'E_L': -70e-3,
            'DeltaT': 2e-3,
            'a': 2e-9,
            'V_T': -50e-3,
            'tau_W': 50e-3,
            'b': 264e-12,
            'V_R': -70e-3,
            'V_cut': -50e-3,
            'n': N_HAIRS_LIST[l],
            'dt': constants['dt']
        }
        
        # Define velocity neuron parameters
        velocity_parameters = {
            'tau': 5e-3,
            'V_T': -50e-3,
            'V_R': -70e-3,
            'n': N_HAIRS_LIST[l],
            'w': WEIGHT_LIST[m],
            'N_input': 1,
            'dt': constants['dt'],
            'refrac': 0,
            'n_sims': 1,
            'multiple_synapses': False
        }
        
        # Initialize hair field
        hair_field = HairField(parameters_hair_field)
        hair_field.get_receptive_field()
        
        # Initialize velocity neuron
        velocity_neuron = LIF(velocity_parameters)
        velocity_neuron.initialize_state()
        
        # Initialize sensory neuron
        sensory_neuron = AdEx(sensory_parameters)
        sensory_neuron.initialize_state()
        
        # Loop through speeds
        for j in range(N_SPEEDS):
            # Create ramp angles
            t1 = np.linspace(0, 180, num=ramp[j])
            t2 = np.linspace(180, 180, num=N_STEPS - ramp[j])
            ramp_angles = np.concatenate((t1, t2))
            hair_angles = hair_field.get_hair_angle(ramp_angles) * I_phi
            
            # Simulate neuron responses
            for i in range(N_STEPS):
                _, spike_1 = sensory_neuron.forward(hair_angles[i])
                _, spike_2 = velocity_neuron.forward(spike_1)
                spike_velocity[i, j] = np.sum(spike_2)
            
            # Calculate spike rates
            spike_rates[l, m, j + 1] = np.sum(spike_velocity[:ramp[j], j] / (ramp[j] * constants['dt']))
        
        # Fit linear regression model to spike rates
        model = LinearRegression().fit(speeds.reshape(-1, 1), spike_rates[l, m, :])
        R_squared = model.score(speeds.reshape(-1, 1), spike_rates[l, m, :])
        
        # Print the slope of the model
        print(rf"$\omega$ = {WEIGHT_LIST[m] * 1000}, $N_h$ = {N_HAIRS_LIST[l]}, slope: {model.coef_}")
        
        # Plot the spike rates
        ax.scatter(0, 0, color=custom_colors[index], marker=custom_markers[index], s=15)
        ax.scatter(speeds, spike_rates[l, m, :], color=custom_colors[index], marker=custom_markers[index], s=15,
                   label=r'$\omega$ = ' + str(WEIGHT_LIST[m] * 1000) + 'mV, $N_{h}$ = ' + str(N_HAIRS_LIST[l]), zorder=2)
        ax.text(speeds[-1] + 12, spike_rates[l, m, -1] - 15 + text_offset[index], fr'$r^2$ = {R_squared:.3f}', color=custom_colors[index])
        
        index += 1  # Increment plot index
    
    # Set plot labels and limits
    ax.set_xlabel(r"Angular velocity (°$\cdot$s$^{-1}$)")
    ax.set_ylabel("Spike rate (Hz)")
    ax.set_xlim([-10, 410])
    ax.set_ylim([-20, 670])
    
    # Add legend to the plot
    ax.legend(frameon=False, bbox_to_anchor=[0.47, 1.04])
    
    # Enable minor ticks
    ax.minorticks_on()
    
    # Adjust layout and save the figure
    fig.tight_layout(pad=parameters['pad'])
    fig.savefig('images/velocity_neuron/P1_fig5a.png')

This cell performs a linear regression on all the velocities to get an r-squared value, intercept and slope.

In [ ]:
# Check if optimization is to be run
if parameters['run_optimization']:
    
    # Initialize the linear regression model
    model = LinearRegression()
    
    # Loop through combinations of hair counts and weights
    for l, m in tqdm(np.ndindex(N_HAIRS_LIST.size, WEIGHT_LIST.size)):
        # Fit the linear regression model to the spike rates
        model.fit(speeds.reshape(-1, 1), spike_rates[l, m, :])
        
        # Calculate the R-squared value for the model
        R_squared = model.score(speeds.reshape(-1, 1), spike_rates[l, m, :])
        
        # Print the R-squared value, model intercept, and coefficients
        print(f"R_squared: {R_squared}")
        print(f"Intercept: {model.intercept_}")
        print(f"Coefficients: {model.coef_}")